In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# OSRM 서버 설정
osrm_server = "http://localhost:5000"
profile = "foot"

# 파일 불러오기
file_path = "골라낸 엣지 거리.xlsx"
edges = pd.read_excel(file_path, engine="openpyxl")

# 도보거리 계산 함수
def get_osrm_distance(start_lon, start_lat, end_lon, end_lat):
    coordinates = f"{start_lon},{start_lat};{end_lon},{end_lat}"
    url = f"{osrm_server}/route/v1/{profile}/{coordinates}?overview=false"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data["routes"][0]["distance"]
        else:
            print(f"Error: {response.status_code} for coordinates {coordinates}")
            return None
    except Exception as e:
        print(f"Exception: {e} for coordinates {coordinates}")
        return None

# tqdm으로 진행 상태 표시
tqdm.pandas()

# 거리 계산 및 새로운 열 추가
edges["edges_distance"] = edges.progress_apply(
    lambda row: get_osrm_distance(
        row["id_start_경도"], row["id_start_위도"], row["id_end_경도"], row["id_end_위도"]
    ),
    axis=1,
)

# 결과 저장
edges.to_excel(file_path, index=False, engine="openpyxl")


100%|██████████| 1247/1247 [00:06<00:00, 186.72it/s]


'edges_distance'가 추가된 데이터가 '골라낸 엣지 거리.xlsx'에 저장되었습니다!
